# Build an interactive map displaying all French museums

#### Open data files can be found here: https://www.data.gouv.fr/fr/datasets/musees-de-france-1/

In [1]:
import pandas as pd
from ipyleaflet import Map, basemaps, basemap_to_tiles, Marker, MarkerCluster
from ipywidgets import Layout
from ipywidgets.embed import embed_minimal_html

In [2]:
fname = "french-museums.csv"
default_zoom = 6
france_barycenter = (46.82, 2.39)
terrain = basemap_to_tiles(basemaps.Stamen.Terrain)

#### <br> Build a pandas dataframe from the data file

In [3]:
df = pd.read_csv(fname, sep=';')
df = df.dropna(subset=['coordonnees_finales'])
df['lat'] = df['coordonnees_finales'].apply(lambda x: str(x).split(',')[0])
df['lon'] = df['coordonnees_finales'].apply(lambda x: str(x).split(',')[1])
df.head(3)

,REGION,DEPARTEMENT,NOM DU MUSEE,REF MUSEE,ADR,CP,VILLE,TELEPHONE1,FAX,SITWEB,FERMETURE ANNUELLE,PERIODE OUVERTURE,JOURS NOCTURNES,DATE APPELLATION,Date retrait appellation par Haut Conseil,coordonnees_finales,coordonnees_ban,coordonnees_cp,lat,lon
0,AUVERGNE-RHÔNE-ALPES,AIN,Musée de la Résistance et de la Déportation de...,0126901,"3, Montée de l'Abbaye",1130,NANTUA,474750750.0,474752758.0,www.musees.ain.fr,"1er mai, lundi de la Pentecôte, 1er novembre","Ouvert du 1er mars au 15 novembre, du mercredi...",NaN,01/02/2003,NaN,"46.152378,5.608744","46.152378,5.608744","46.1948352393,5.7148151411",46.152378,5.608744
1,AUVERGNE-RHÔNE-ALPES,AIN,Musée de la Dombes,0144301,Maison de la Dombes,1330,VILLARS DE LA DOMBES,NaN,NaN,http://www.museedeladombes.fr/,NaN,NaN,NaN,01/02/2003,NaN,"46.000448,5.026912","46.000448,5.026912","45.9976909227,5.02164504767",46.000448,5.026912
2,CENTRE-VAL DE LOIRE,EURE ET LOIR,Musée des Beaux-Arts et d'Histoire Naturelle,2808801,"3, rue Toufaire",28200,CHATEAUDUN,237455536.0,237455446.0,www.ville-chateaudun.fr ou www.musees.regionce...,"1er janvier, 1er mai et 25 décembre","Ouvert d'octobre à mai du lundi au vendredi, d...",NaN,01/02/2003,NaN,"48.070792,1.327905","48.070792,1.327905","48.0750607367,1.3674487976",48.070792,1.327905


#### <br> Display the size of the table (missing museums that are not geolocalized) + the different fields

In [4]:
print(f'size of the table: {df.shape}')
print('Fields of the table:')
df.columns

size of the table: (1153, 20)
Fields of the table:


Index(['REGION', 'DEPARTEMENT', 'NOM DU MUSEE', 'REF MUSEE', 'ADR', 'CP',
       'VILLE', 'TELEPHONE1', 'FAX', 'SITWEB', 'FERMETURE ANNUELLE',
       'PERIODE OUVERTURE', 'JOURS NOCTURNES', 'DATE APPELLATION',
       'Date retrait appellation par Haut Conseil', 'coordonnees_finales',
       'coordonnees_ban', 'coordonnees_cp', 'lat', 'lon'],
      dtype='object')

#### <br> Build a map with ipyleaflet

In [5]:
m = Map(layers=(terrain, ), center=france_barycenter, zoom=default_zoom, layout=Layout(width='100%', height='900px'))
marker_list = []
for i in range(df.shape[0]):
    s = df.iloc[i]
    marker_list.append(Marker(location=(s.lat, s.lon), draggable=False, title=s['NOM DU MUSEE']))
marker_cluster = MarkerCluster(markers=marker_list)
m.add_layer(marker_cluster)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

#### <br> Save the map as an interactive HTML page

In [6]:
embed_minimal_html('french_museums.html', views=[m], title='French museums')

#### <br> Basic stats: cities with most museums

In [9]:
df['VILLE'].value_counts().nlargest(10)

PARIS              51
MARSEILLE          15
NICE               11
STRASBOURG         10
LYON               10
TOULOUSE            8
ROUEN               8
AIX-EN-PROVENCE     7
TROYES              7
BORDEAUX            7
Name: VILLE, dtype: int64